In [7]:
import math
from sklearn import neighbors, svm
import os
import os.path
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
import cv2
from matplotlib.pyplot import imshow 

In [8]:
train_dir = "ActorsDataset"

trainX = []
trainY = []
testX = []
testY = []

In [9]:
cell_size = (16, 16)  # h x w in pixels
block_size = (2, 2)  # h x w in cells
nbins = 9  # number of orientation bins

hog = cv2.HOGDescriptor(_winSize=(64 // cell_size[1] * cell_size[1],
                          64 // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)

for class_dir in os.listdir(train_dir):

    nr = 0
    for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
        
        image = face_recognition.load_image_file(img_path)
        face_bounding_boxes = face_recognition.face_locations(image)
        
        if len(face_bounding_boxes) == 1:
            (top, right, bottom, left) = face_bounding_boxes[0]
            face_img = cv2.resize(image[top:bottom, left:right],(64,64))
            img_hog = hog.compute(face_img)
            #imshow(face_img)
            #nx, ny = trainX[1].shape
            #d2_train_dataset = trainX[0].reshape(nx*ny)
            if nr < 20:
                testX.append(img_hog.reshape(324))
                testY.append(class_dir)
            else:
                trainX.append(img_hog.reshape(324))
                trainY.append(class_dir)
            nr = nr + 1

In [10]:
print(len(trainX))
print(len(trainY))
print(len(testX))
print(len(testY))

8654
8654
2000
2000


In [27]:
def trainKNN(model_save_path=None, n_neighbors=None, knn_algo='ball_tree'):
    
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(trainX))))

    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(trainX, trainY)

    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf

In [28]:
print("Training KNN classifier...")
knn_clf = trainKNN(n_neighbors=5)
print("Training complete!")

Training KNN classifier...
Training complete!


In [29]:
def predictKNN(faces_encodings, distance_threshold=0.6):

    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][0][0] <= distance_threshold]
    
    if are_matches:
        return knn_clf.predict(faces_encodings)[0]
    else:
        return "unkown"

In [30]:
total = 0
nrCorecte = 0
for i in range(len(testX)):
    name = predictKNN([testX[i]])
    total = total + 1
    if name == testY[i]:
        nrCorecte = nrCorecte + 1
    else:
        print("Corect: " + testY[i] + "; Prezis: " + name)

print("Corecte KNN: " + str(nrCorecte))
print("Totale KNN: " + str(total))
print("Acc KNN: " + str(nrCorecte/total))

Corect: Madchen Amick; Prezis: Karen Gillan
Corect: Madchen Amick; Prezis: Robin Wright
Corect: Madchen Amick; Prezis: Serinda Swan
Corect: Madchen Amick; Prezis: Gal Gadot
Corect: Madchen Amick; Prezis: Alison Brie
Corect: Madchen Amick; Prezis: Jessica Henwick
Corect: Madchen Amick; Prezis: Shailene Woodley
Corect: Madchen Amick; Prezis: Travis Fimmel
Corect: Madchen Amick; Prezis: Jessica Henwick
Corect: Madchen Amick; Prezis: Brie Larson
Corect: Madchen Amick; Prezis: Annabelle Wallis
Corect: Madchen Amick; Prezis: Connie Nielsen
Corect: Madchen Amick; Prezis: Margot Robbie
Corect: Madchen Amick; Prezis: Charlize Theron
Corect: Madchen Amick; Prezis: Alison Brie
Corect: Madchen Amick; Prezis: Scarlett Johansson
Corect: Madchen Amick; Prezis: Johnny Depp
Corect: Madchen Amick; Prezis: Charlize Theron
Corect: Jessica Henwick; Prezis: Emma Watson
Corect: Jessica Henwick; Prezis: Haley Bennett
Corect: Jessica Henwick; Prezis: Pom Klementieff
Corect: Jessica Henwick; Prezis: Nina Dobrev

Corect: Travis Fimmel; Prezis: Dave Bautista
Corect: Travis Fimmel; Prezis: Keanu Reeves
Corect: Ryan Gosling; Prezis: Tom Cruise
Corect: Ryan Gosling; Prezis: Alexander Skarsgard
Corect: Ryan Gosling; Prezis: Charlie Hunnam
Corect: Ryan Gosling; Prezis: Emily Browning
Corect: Ryan Gosling; Prezis: Bill Paxton
Corect: Ryan Gosling; Prezis: Tom Hardy
Corect: Ryan Gosling; Prezis: Charlie Hunnam
Corect: Ryan Gosling; Prezis: Dan Stevens
Corect: Ryan Gosling; Prezis: Tom Cruise
Corect: Ryan Gosling; Prezis: Chris Pratt
Corect: Ryan Gosling; Prezis: Dave Bautista
Corect: Ryan Gosling; Prezis: Tom Hardy
Corect: Ryan Gosling; Prezis: Sophia Lillis
Corect: Elizabeth Debicki; Prezis: Rebecca Ferguson
Corect: Elizabeth Debicki; Prezis: Pom Klementieff
Corect: Elizabeth Debicki; Prezis: Scarlett Johansson
Corect: Elizabeth Debicki; Prezis: Johnny Depp
Corect: Elizabeth Debicki; Prezis: Emma Watson
Corect: Elizabeth Debicki; Prezis: Margot Robbie
Corect: Elizabeth Debicki; Prezis: Allison William

Corect: Jason Momoa; Prezis: Chris Hemsworth
Corect: Jason Momoa; Prezis: Charlie Hunnam
Corect: Jason Momoa; Prezis: Tom Hardy
Corect: Jason Momoa; Prezis: Dan Stevens
Corect: Jason Momoa; Prezis: Charlie Hunnam
Corect: Jason Momoa; Prezis: Madchen Amick
Corect: Jason Momoa; Prezis: Leonardo Dicaprio
Corect: Zoe Saldana; Prezis: Gal Gadot
Corect: Zoe Saldana; Prezis: Amy Adams
Corect: Zoe Saldana; Prezis: Chris Pine
Corect: Zoe Saldana; Prezis: Chris Pratt
Corect: Zoe Saldana; Prezis: Connie Nielsen
Corect: Zoe Saldana; Prezis: Sophia Lillis
Corect: Zoe Saldana; Prezis: Shailene Woodley
Corect: Zoe Saldana; Prezis: Kate Beckinsale
Corect: Zoe Saldana; Prezis: Ana de Armas
Corect: Zoe Saldana; Prezis: Luke Evans
Corect: Zoe Saldana; Prezis: Anya Taylor-Joy
Corect: Zoe Saldana; Prezis: Scarlett Johansson
Corect: Zoe Saldana; Prezis: Sophie Turner
Corect: Zoe Saldana; Prezis: Haley Bennett
Corect: Zoe Saldana; Prezis: Chloe Grace Moretz
Corect: Zoe Saldana; Prezis: Margot Robbie
Corect: 

Corect: Emma Watson; Prezis: Charlie Hunnam
Corect: Ana de Armas; Prezis: Rose Leslie
Corect: Ana de Armas; Prezis: Julia Garner
Corect: Ana de Armas; Prezis: Kate Beckinsale
Corect: Ana de Armas; Prezis: Emily Browning
Corect: Ana de Armas; Prezis: Tom Hardy
Corect: Ana de Armas; Prezis: Jodie Whittaker
Corect: Ana de Armas; Prezis: Margot Robbie
Corect: Ana de Armas; Prezis: Rose Leslie
Corect: Ana de Armas; Prezis: Scarlett Johansson
Corect: Ana de Armas; Prezis: Daisy Ridley
Corect: Ana de Armas; Prezis: Dakota Johnson
Corect: Ana de Armas; Prezis: Katherine Waterston
Corect: Ana de Armas; Prezis: Alexander Skarsgard
Corect: Jodie Whittaker; Prezis: Kate Beckinsale
Corect: Jodie Whittaker; Prezis: Allison Williams
Corect: Jodie Whittaker; Prezis: Emily Browning
Corect: Jodie Whittaker; Prezis: Charlie Hunnam
Corect: Jodie Whittaker; Prezis: Rebecca Ferguson
Corect: Jodie Whittaker; Prezis: Robin Wright
Corect: Jodie Whittaker; Prezis: Teresa Palmer
Corect: Jodie Whittaker; Prezis: 

Corect: Britt Robertson; Prezis: Zoey Deutch
Corect: Britt Robertson; Prezis: Alexandra Daddario
Corect: Britt Robertson; Prezis: Bella Thorne
Corect: Britt Robertson; Prezis: Rose Leslie
Corect: Britt Robertson; Prezis: Kate Beckinsale
Corect: Britt Robertson; Prezis: Rosamund Pike
Corect: Britt Robertson; Prezis: Haley Bennett
Corect: Britt Robertson; Prezis: Dakota Johnson
Corect: Britt Robertson; Prezis: Yvonne Strahovski
Corect: Britt Robertson; Prezis: Finn Jones
Corect: Britt Robertson; Prezis: Scarlett Johansson
Corect: Millie Bobby Brown; Prezis: Travis Fimmel
Corect: Millie Bobby Brown; Prezis: Hugh Jackman
Corect: Millie Bobby Brown; Prezis: Emily Browning
Corect: Millie Bobby Brown; Prezis: Luke Evans
Corect: Millie Bobby Brown; Prezis: Katheryn Winnick
Corect: Millie Bobby Brown; Prezis: Chris Pine
Corect: Millie Bobby Brown; Prezis: Amy Adams
Corect: Millie Bobby Brown; Prezis: Tom Hardy
Corect: Millie Bobby Brown; Prezis: Chris Pine
Corect: Millie Bobby Brown; Prezis: Sh

Corect: Katheryn Winnick; Prezis: Felicity Jones
Corect: Katheryn Winnick; Prezis: Annabelle Wallis
Corect: Katheryn Winnick; Prezis: Madchen Amick
Corect: Katheryn Winnick; Prezis: Sophia Lillis
Corect: Katheryn Winnick; Prezis: Margot Robbie
Corect: Katheryn Winnick; Prezis: Dave Bautista
Corect: Katheryn Winnick; Prezis: Tom Cruise
Corect: Katheryn Winnick; Prezis: Robin Wright
Corect: Katheryn Winnick; Prezis: Lily James
Corect: Katheryn Winnick; Prezis: Bill Paxton
Corect: Katheryn Winnick; Prezis: Julia Garner
Corect: Katheryn Winnick; Prezis: Connie Nielsen
Corect: Katheryn Winnick; Prezis: Kit Harington
Corect: Katheryn Winnick; Prezis: Teresa Palmer
Corect: Cole Sprouse; Prezis: Kate Mara
Corect: Cole Sprouse; Prezis: Katherine Waterston
Corect: Cole Sprouse; Prezis: Kelly Rohrbach
Corect: Cole Sprouse; Prezis: Finn Jones
Corect: Cole Sprouse; Prezis: Luke Evans
Corect: Cole Sprouse; Prezis: Dylan Minnette
Corect: Cole Sprouse; Prezis: Adrianne Palicki
Corect: Cole Sprouse; Pr

Corect: Christian Navarro; Prezis: Sophia Lillis
Corect: Annabelle Wallis; Prezis: Chloe Grace Moretz
Corect: Annabelle Wallis; Prezis: Laura Haddock
Corect: Annabelle Wallis; Prezis: Amy Adams
Corect: Annabelle Wallis; Prezis: Amy Adams
Corect: Annabelle Wallis; Prezis: Zoey Deutch
Corect: Annabelle Wallis; Prezis: Adrianne Palicki
Corect: Annabelle Wallis; Prezis: Daisy Ridley
Corect: Annabelle Wallis; Prezis: Tom Holland
Corect: Annabelle Wallis; Prezis: Dylan Minnette
Corect: Annabelle Wallis; Prezis: Dakota Johnson
Corect: Annabelle Wallis; Prezis: Rosamund Pike
Corect: Annabelle Wallis; Prezis: Bella Thorne
Corect: Annabelle Wallis; Prezis: Chloe Grace Moretz
Corect: Annabelle Wallis; Prezis: Rebecca Ferguson
Corect: Julia Garner; Prezis: Jennifer Lawrence
Corect: Julia Garner; Prezis: Felicity Jones
Corect: Julia Garner; Prezis: Eiza Gonzalez
Corect: Julia Garner; Prezis: Connie Nielsen
Corect: Julia Garner; Prezis: Halston Sage
Corect: Julia Garner; Prezis: Jennifer Lawrence
Co

Corect: Chloe Grace Moretz; Prezis: Emma Watson
Corect: Chloe Grace Moretz; Prezis: Elodie Yung
Corect: Chloe Grace Moretz; Prezis: Madchen Amick
Corect: Chloe Grace Moretz; Prezis: Brie Larson
Corect: Chloe Grace Moretz; Prezis: Katheryn Winnick
Corect: Chloe Grace Moretz; Prezis: Jessica Henwick
Corect: Chloe Grace Moretz; Prezis: Gal Gadot
Corect: Chloe Grace Moretz; Prezis: Allison Williams
Corect: Lena Headey; Prezis: Yvonne Strahovski
Corect: Lena Headey; Prezis: Bella Thorne
Corect: Lena Headey; Prezis: Gal Gadot
Corect: Lena Headey; Prezis: Sean Young
Corect: Lena Headey; Prezis: Allison Williams
Corect: Lena Headey; Prezis: Robin Wright
Corect: Lena Headey; Prezis: Christian Navarro
Corect: Lena Headey; Prezis: Charlie Hunnam
Corect: Lena Headey; Prezis: Shailene Woodley
Corect: Lena Headey; Prezis: Margot Robbie
Corect: Lena Headey; Prezis: James McAvoy
Corect: Lena Headey; Prezis: Haley Bennett
Corect: Lena Headey; Prezis: Zoey Deutch
Corect: Charlize Theron; Prezis: Zoe Sal

In [63]:
def trainSVM(model_save_path=None):

    svm_clf = svm.SVC(kernel='linear', C = 15.0)
    svm_clf.fit(trainX, trainY)
    
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(svm_clf, f)

    return svm_clf

In [64]:
print("Training SVM classifier...")
svm_clf = trainSVM()
print("Training complete!")

Training SVM classifier...
Training complete!


In [65]:
def predictSVM(faces_encodings):
    return svm_clf.predict(faces_encodings)[0]

In [66]:
total = 0
nrCorecte = 0
for i in range(len(testX)):
    name = predictSVM([testX[i]])
    total = total + 1
    if name == testY[i]:
        nrCorecte = nrCorecte + 1
    else:
        print("Corect: " + testY[i] + "; Prezis: " + name)

print("Corecte KNN: " + str(nrCorecte))
print("Totale KNN: " + str(total))
print("Acc KNN: " + str(nrCorecte/total))

Corect: Madchen Amick; Prezis: Katherine Langford
Corect: Madchen Amick; Prezis: Jodie Whittaker
Corect: Madchen Amick; Prezis: Charlize Theron
Corect: Madchen Amick; Prezis: Johnny Depp
Corect: Madchen Amick; Prezis: Anya Taylor-Joy
Corect: Madchen Amick; Prezis: Lily James
Corect: Madchen Amick; Prezis: Kaya Scodelario
Corect: Madchen Amick; Prezis: Lena Headey
Corect: Madchen Amick; Prezis: Bryce Dallas Howard
Corect: Madchen Amick; Prezis: Chris Pine
Corect: Madchen Amick; Prezis: Julia Garner
Corect: Madchen Amick; Prezis: Lily James
Corect: Jessica Henwick; Prezis: Chloe Grace Moretz
Corect: Jessica Henwick; Prezis: Rosamund Pike
Corect: Jessica Henwick; Prezis: Eiza Gonzalez
Corect: Jessica Henwick; Prezis: Gal Gadot
Corect: Jessica Henwick; Prezis: Alexander Skarsgard
Corect: Jessica Henwick; Prezis: Emily Browning
Corect: Jessica Henwick; Prezis: Kelly Rohrbach
Corect: Jessica Henwick; Prezis: Alicia Vikander
Corect: Jessica Henwick; Prezis: Bill Skarsgard
Corect: Jessica Henw

Corect: Tom Hardy; Prezis: Alexander Skarsgard
Corect: Tom Hardy; Prezis: Kit Harington
Corect: Tom Hardy; Prezis: Dave Bautista
Corect: Ruby Rose; Prezis: Karen Gillan
Corect: Ruby Rose; Prezis: Bella Thorne
Corect: Ruby Rose; Prezis: Ana de Armas
Corect: Ruby Rose; Prezis: Alison Brie
Corect: Ruby Rose; Prezis: Zendaya
Corect: Ruby Rose; Prezis: Tom Cruise
Corect: Elodie Yung; Prezis: Bella Thorne
Corect: Elodie Yung; Prezis: Haley Bennett
Corect: Elodie Yung; Prezis: Scarlett Johansson
Corect: Elodie Yung; Prezis: Rosamund Pike
Corect: Elodie Yung; Prezis: Emilia Clarke
Corect: Elodie Yung; Prezis: Ruby Rose
Corect: Elodie Yung; Prezis: Sofia Boutella
Corect: Elodie Yung; Prezis: Alexandra Daddario
Corect: Elodie Yung; Prezis: Dafne Keen
Corect: Gal Gadot; Prezis: Mary Elizabeth Winstead
Corect: Gal Gadot; Prezis: Jessica Henwick
Corect: Gal Gadot; Prezis: Ruby Rose
Corect: Gal Gadot; Prezis: Dafne Keen
Corect: Gal Gadot; Prezis: Kate Mara
Corect: Gal Gadot; Prezis: Abigail Breslin


Corect: Lily James; Prezis: Adrianne Palicki
Corect: Lily James; Prezis: Madchen Amick
Corect: Lily James; Prezis: Brie Larson
Corect: Lily James; Prezis: Elodie Yung
Corect: Lily James; Prezis: Katheryn Winnick
Corect: Lily James; Prezis: Finn Jones
Corect: Lily James; Prezis: Abigail Breslin
Corect: Kaya Scodelario; Prezis: Dafne Keen
Corect: Kaya Scodelario; Prezis: Katheryn Winnick
Corect: Kaya Scodelario; Prezis: Pom Klementieff
Corect: Kaya Scodelario; Prezis: James McAvoy
Corect: Kaya Scodelario; Prezis: Rebecca Ferguson
Corect: Kaya Scodelario; Prezis: Eiza Gonzalez
Corect: Kaya Scodelario; Prezis: Madchen Amick
Corect: Kaya Scodelario; Prezis: Eiza Gonzalez
Corect: Kaya Scodelario; Prezis: Rebecca Ferguson
Corect: Kaya Scodelario; Prezis: Emilia Clarke
Corect: Kaya Scodelario; Prezis: Dafne Keen
Corect: Auli_i Cravalho; Prezis: Zoe Saldana
Corect: Auli_i Cravalho; Prezis: Annabelle Wallis
Corect: Auli_i Cravalho; Prezis: Kaya Scodelario
Corect: Auli_i Cravalho; Prezis: Kelly R

Corect: Katheryn Winnick; Prezis: Teresa Palmer
Corect: Katheryn Winnick; Prezis: Julia Garner
Corect: Katheryn Winnick; Prezis: Teresa Palmer
Corect: Katheryn Winnick; Prezis: Emily Browning
Corect: Cole Sprouse; Prezis: Brie Larson
Corect: Cole Sprouse; Prezis: Chloe Grace Moretz
Corect: Cole Sprouse; Prezis: James McAvoy
Corect: Cole Sprouse; Prezis: Dylan Minnette
Corect: Cole Sprouse; Prezis: Adrianne Palicki
Corect: Cole Sprouse; Prezis: Charlie Hunnam
Corect: Cole Sprouse; Prezis: Annabelle Wallis
Corect: Cole Sprouse; Prezis: Dave Bautista
Corect: Cole Sprouse; Prezis: Sean Young
Corect: Cole Sprouse; Prezis: Sylvester Stallone
Corect: Cole Sprouse; Prezis: Tom Cruise
Corect: Cole Sprouse; Prezis: Charlize Theron
Corect: Cole Sprouse; Prezis: Katherine Langford
Corect: Cole Sprouse; Prezis: Rose Leslie
Corect: Cole Sprouse; Prezis: Kit Harington
Corect: James McAvoy; Prezis: Leonardo Dicaprio
Corect: James McAvoy; Prezis: Jason Momoa
Corect: James McAvoy; Prezis: Tom Hardy
Core

Corect: Kit Harington; Prezis: Finn Jones
Corect: Daisy Ridley; Prezis: Abigail Breslin
Corect: Daisy Ridley; Prezis: Eiza Gonzalez
Corect: Daisy Ridley; Prezis: Cole Sprouse
Corect: Daisy Ridley; Prezis: Annabelle Wallis
Corect: Daisy Ridley; Prezis: Madchen Amick
Corect: Daisy Ridley; Prezis: Allison Williams
Corect: Daisy Ridley; Prezis: Kate Mara
Corect: Daisy Ridley; Prezis: Katheryn Winnick
Corect: Daisy Ridley; Prezis: Madchen Amick
Corect: Daisy Ridley; Prezis: Allison Williams
Corect: Daisy Ridley; Prezis: Nina Dobrev
Corect: Daisy Ridley; Prezis: Nina Dobrev
Corect: Daisy Ridley; Prezis: Lily James
Corect: Kelly Rohrbach; Prezis: Emma Stone
Corect: Kelly Rohrbach; Prezis: Rebecca Ferguson
Corect: Kelly Rohrbach; Prezis: Cara Delevingne
Corect: Kelly Rohrbach; Prezis: Emily Browning
Corect: Kelly Rohrbach; Prezis: Zoey Deutch
Corect: Kelly Rohrbach; Prezis: Karen Gillan
Corect: Kelly Rohrbach; Prezis: Teresa Palmer
Corect: Kelly Rohrbach; Prezis: Cara Delevingne
Corect: Kelly 